In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import duckdb
import boto3
import os
import pandas as pd
from pathlib import Path

import src.locations as loc

HTTPFS_PATH = loc.PROJECT_DIR / 'scratch' / 'httpfs.duckdb_extension'

In [3]:
client = boto3.client('s3', region_name=os.environ['S3_REGION']) 
response = client.get_object(
    Bucket='jupyter.notebook.uktrade.io', 
    Key=os.environ['S3_PREFIX_TEAM_DDAT_DATA_SCIENCE'] + '.tmp/dummy.csv'
) 
df = pd.read_csv(response['Body'])
df

,foo,bar
0,a,1
1,b,2
2,c,3


In [10]:
con = duckdb.connect()

In [11]:
# via https://duckdb.org/docs/extensions/httpfs.html

con.query(f"""
    install '{HTTPFS_PATH.resolve()}';
    load '{HTTPFS_PATH.resolve()}';
    set s3_region='{os.environ['S3_REGION']}';
""")

In [13]:
'/'.join([
    os.environ['S3_BUCKET'],
    os.environ['S3_PREFIX_TEAM_DDAT_DATA_SCIENCE'],
    '.tmp',
    'dummy.csv'
])

'jupyter.notebook.uktrade.io/teams/_team_ddat_data_science//.tmp/dummy.csv'

Try this: https://stackoverflow.com/questions/74789412/aws-role-vs-iam-credential-in-duckdb-httpfs-call

In [12]:
con.query(f"""
    select *
    from 's3://{
        '/'.join([
            os.environ['S3_BUCKET'],
            os.environ['S3_PREFIX_TEAM_DDAT_DATA_SCIENCE'],
            '.tmp',
            'dummy.csv'
        ])
    }';
""")

IOException: IO Error: Unable to connect to URL "s3://jupyter.notebook.uktrade.io/teams/_team_ddat_data_science//.tmp/dummy.csv": 403 (Forbidden)

In [4]:
df = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})

In [9]:
con.query("""
    select
        *
    from
        df
""")

┌───────┬───────┐
│ col1  │ col2  │
│ int64 │ int64 │
├───────┼───────┤
│     1 │     3 │
│     2 │     4 │
└───────┴───────┘

In [ ]:
os.environ['S3_HOST']
os.environ['S3_BUCKET']

In [33]:
[k for k in list(dict(os.environ).keys()) if 's3' in k.lower()]

['S3_HOST',
 'S3_BUCKET',
 'S3_REGION',
 'S3_PREFIX_TEAM_DDAT_DATA_SCIENCE',
 'S3_PREFIX_TEAM_DEET',
 'S3_PREFIX']

In [35]:
[k for k in list(dict(os.environ).keys()) if 'token' in k.lower()]

['MLFLOW_TRACKING_TOKEN']

In [42]:
[k for k in list(dict(os.environ).keys()) if 'aws' in k.lower()]

['AWS_CONTAINER_CREDENTIALS_RELATIVE_URI',
 'AWS_EXECUTION_ENV',
 'AWS_DEFAULT_REGION',
 'AWS_REGION']

In [14]:
# os.environ['AWS_CONTAINER_CREDENTIALS_RELATIVE_URI']
# os.environ['ECS_CONTAINER_METADATA_URI_V4']